In [1]:
import pandas as pd
import json
import re
from transformers import T5Tokenizer
import nltk
nltk.download('punkt')

2023-03-26 15:50:21.865176: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 15:50:22.543802: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/usr/local/lib:/usr/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cud

True

In [2]:
df = pd.read_excel('../WOT_extractive_original.xlsx').drop(columns=['Unnamed: 0']) 
df['input'] = df['input'].apply(lambda x: str(x).replace('|', '| ')) 
#df['input'] = df['input'].apply(lambda x: str(x).replace('\n', '\n '))
df['input'] = df['input'].apply(lambda x: str(x).replace(';', '; ')) 
df['output'] = df['output'].apply(lambda x: str(x).replace('|', '| ')) 
df['output'] = df['output'].apply(lambda x: str(x).replace('\\n', '\n')) 
df['output'] = df['output'].apply(lambda x: str(x).replace(';', '; ')) 

In [3]:
print(df.loc[674, 'output'])
df.loc[674, 'output'] = 'use a dry cloth to soak up any liquid from the vomit'

use a dry cloth to soak up any liquid from the vomi


# Perfect Context

In [4]:
model_name = "allenai/unifiedqa-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

def compare(a, b):
    for i in range(len(b)):
        if a[i] != b[i]:
            return False
    return True

def get_trunc_instructions(instructions, answer, available_tokens):
    if compare(answer, [148, 33, 612]):
        if len(instructions) < available_tokens:
            return instructions
        return instructions[-available_tokens:]
    if compare(answer, [4273]):
        if len(instructions) < available_tokens:
            return instructions
        return instructions[-available_tokens:]
    if compare(answer, [150]):
        if len(instructions) < available_tokens:
            return instructions
        return instructions[-available_tokens:]
    if compare(answer, [766, 8, 2912, 19, 344, 9455, 11, 3, 4013, 3, 1956, 89, 41, 2884, 11, 944, 3, 1956, 75, 61]):
        answer = [766, 8, 2912, 19, 344, 9455, 11, 3, 4013, 3, 1956, 89, 41, 2884, 11, 944, 3, 1956, 75, 137]
    if compare(answer, [314, 716, 41, 127, 1200, 6, 3, 99, 25, 114, 61]):
        answer = [314, 716, 41, 127, 1200, 6, 3, 99, 25, 114, 201]
    if compare(answer, [3, 99, 338, 5726, 3, 11049, 3321, 7, 6, 18874, 16, 387]):
        answer = [41, 99, 338, 5726, 3, 11049, 3321, 7, 6, 18874, 16, 387]
    if compare(answer, [3, 20987, 371, 3224, 2912]):
        answer = [41, 20987, 371, 3224, 2912]
    if compare(answer, [6273, 3380, 49, 342, 21, 44, 709, 505, 716, 41, 127, 95, 12, 204, 477, 61]):
        answer = [6273, 3380, 49, 342, 21, 44, 709, 505, 716, 41, 127, 95, 12, 204, 477, 137]
    
    try:
        answer = answer[:-1]
        position = [(i, i+len(answer)) for i in range(len(instructions)) if compare(instructions[i:i+len(answer)],answer)]
        start = position[0][0]
        end = position[0][1]
    except:
        print(tokenizer.decode(answer))
        print(tokenizer.decode(instructions))
        print(answer)
        print(instructions)
        
    
    before = True
    
    while end-start < available_tokens:
        if before:
            before = False
            start -= 10
        else:
            before = True
            end += 10
            
        start = max(start, 0)
        end = min(len(instructions), end)
        
        if start==0 and end ==len(instructions):
            return instructions[start:end]
    
    while end-start < available_tokens:
        if before:
            before = False
            start -= 1
        else:
            before = True
            end += 1
            
        start = max(start, 0)
        end = min(len(instructions), end)
        
        if start==0 and end ==len(instructions):
            return instructions[start:end]
        
    return instructions[start:end]
        
    

df['tokens_taken'] = df.apply(lambda row: len(tokenizer.encode(f"{row['question']}SPLITInstructions: \n\nHistory: {row['history']})")), axis=1)
df['encoded_instructions'] = df.apply(lambda row: tokenizer.encode(row['input'][len(f"{row['question']}SPLITInstructions: ") : len(row['input']) - 2 - len(f"\n\nHistory: {row['history']})")]), axis=1)
df['trunc_instructions'] = df.apply(lambda row: get_trunc_instructions(row['encoded_instructions'], tokenizer.encode(row['output']), 512 - row['tokens_taken']), axis=1)
df['trunc_instructions'] = df['trunc_instructions'].apply(lambda x: tokenizer.decode(x[:-1]))
df['input_context_selected'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:{row['trunc_instructions']}\nHistory:{row['history']}", axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


In [5]:
df.to_excel('../WOT_extractive_final.xlsx')


## SBERT Retrieval
https://www.sbert.net/

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

## get embeddings

In [7]:
#df['steps'] = df['steps'].apply(lambda x: x[2:-2].split("\', \'"))
df['steps_embeddings'] = df['steps'].apply(model.encode)
df['question_embedding'] = df['question'].apply(lambda x: model.encode([x])[0])

## consine similarity

In [8]:
df['relevant_step_ind'] = df.apply(lambda row: util.cos_sim(row['question_embedding'], row['steps_embeddings']).argmax(), axis=1)
df['relevant_step'] = df.apply(lambda row: row['steps'][row['relevant_step_ind']], axis=1)
df['context_SBERT'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:\n|Title:\n{row['title']}\n|Ingredients:\n{row['ingredients']}\n|Step:\n{row['relevant_step']}\n|History:\n{row['history']}", axis=1)

In [9]:
df.to_excel('../WOT_extractive_final.xlsx')

# Vary number of Utterances in conversation History

In [10]:
df_cooking = pd.read_json('https://wizard-of-tasks.s3.us-west-2.amazonaws.com/wizard_of_tasks_cooking_v1.0.json').transpose()
df_diy = pd.read_json('https://wizard-of-tasks.s3.us-west-2.amazonaws.com/wizard_of_tasks_diy_v1.0.json').transpose()

df_hist = pd.concat([df_cooking, df_diy])

def get_history(conversation, question, number_of_utterances):
    conv = df_hist.loc[conversation, 'turns']
    for i, el in enumerate([el['text'] for el in conv]):
        if el == question:
            ind = i
            break
    
    out = [f"{el['role']}: {el['text']}" for el in conv[max(0, ind-number_of_utterances):ind]]

    return ' | '.join(out)


In [11]:
df['history_6'] = df.apply(lambda row: get_history(row['conversation'], row['question'], 6), axis=1)
df['tokens_taken_6'] = df.apply(lambda row: len(tokenizer.encode(f"{row['question']}SPLITInstructions: \n\nHistory: {row['history_6']})")), axis=1)
df['encoded_instructions_6'] = df.apply(lambda row: tokenizer.encode(row['input'][len(f"{row['question']}SPLITInstructions: ") : len(row['input']) - 1 - len(f"\n\nHistory: {row['history']})")]), axis=1)
df['trunc_instructions_6'] = df.apply(lambda row: get_trunc_instructions(row['encoded_instructions_6'], tokenizer.encode(row['output']), 512 - row['tokens_taken_6']), axis=1)
df['trunc_instructions_6'] = df['trunc_instructions_6'].apply(lambda x: tokenizer.decode(x[:-1]))

df['input_context_selected_0_history'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:{row['trunc_instructions_6']}", axis=1)

df['history_2'] = df.apply(lambda row: get_history(row['conversation'], row['question'], 2), axis=1)
df['input_context_selected_2_history'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:{row['trunc_instructions_6']}\nHistory:{row['history_2']}", axis=1)

df['history_4'] = df.apply(lambda row: get_history(row['conversation'], row['question'], 4), axis=1)
df['input_context_selected_4_history'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:{row['trunc_instructions_6']}\nHistory:{row['history_4']}", axis=1)


df['input_context_selected_6_history'] = df.apply(lambda row: f"{row['question']}SPLITInstructions:{row['trunc_instructions_6']}\nHistory:{row['history_6']}", axis=1)

In [14]:
print(df.loc[12, 'input_context_selected_6_history'])

Just to clarify did you mean to say I should add red pepper flakes?SPLITInstructions:a little heat from red pepper flakes. | Ingredients: 1 pound dried rigatoni pasta 1/4 teaspoon fine sea salt 3 tablespoons extra-virgin olive oil 1 large yellow onion, thinly sliced 3 cloves garlic, finely chopped 1/4 teaspoon ground black pepper 1/4 teaspoon crushed red chile flakes 1/2 cup raw walnut halves and pieces 1 (28.0-ounce) can whole tomatoes, drained 2 (6.0-ounce) cans tuna packed in water, drained 3/4 cup seedless raisins 1/2 cup fresh flat-leaf parsley, chopped | Steps: cook pasta in a 6-quart pot of boiling salted water according to package directions until al dente. meanwhile, heat oil in a large skillet over high heat until hot but not smoking. add onions, garlic, salt, pepper and red pepper flakes and cook, stirring occasionally, until golden, 6 to 8 minutes. add walnuts and cook for 2 to 3 minutes, until lightly toasted. add tomatoes, breaking them up with a spoon, and simmer for 5 m

In [12]:
df.to_excel('../WOT_extractive_final.xlsx')